### Calculate error rate of prediction of different model

In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [12]:
veg2 = ['香蕉','洋香瓜-網狀紅肉','蓮霧-紅蓮霧','鳳梨-金鑽鳳梨','南瓜-木瓜形',
        '大蒜-蒜仁','青蔥-日蔥','小白菜-土白菜','芽菜類-黃豆牙','木瓜-網室紅肉']

models = ['LSTM1','LSTM2','ARIMA','Prophet']

In [107]:
# Change vegetable class id
idx = 9
###########################
dataset = pd.read_csv('./All_Predict/All_Predict_'+veg2[idx]+'_'+'Price.csv')

In [108]:
dataset

,date,Price_9,LSTM1_9,LSTM2_9,LSTM2_fs_9,ARIMA_9,Prophet_9
0,2019-01-01,26.0,22.861876,19.536448,25.110641,22.901349,23.414170
1,2019-01-02,31.3,19.730017,21.405718,25.642231,23.896869,23.178763
2,2019-01-03,30.2,19.985209,24.893736,26.416279,24.435881,22.958030
3,2019-01-04,30.4,22.033504,28.010067,26.897085,24.500020,22.250773
4,2019-01-05,29.8,25.413652,30.301266,26.824453,24.168757,21.771556
...,...,...,...,...,...,...,...
293,2019-12-25,34.5,30.293420,23.511997,25.035316,39.363605,16.763881
294,2019-12-26,32.9,30.398933,25.466755,25.472092,33.826128,16.199832
295,2019-12-27,33.2,28.251450,22.962618,25.321056,33.413738,15.178269
296,2019-12-28,32.8,26.129517,21.515228,25.469582,33.473035,14.416234


In [109]:
# fill in prophet missing value
for i in range(len(dataset)):
    if pd.isnull(dataset['Prophet_'+str(idx)][i]):
        print(dataset['date'][i],dataset['Prophet_'+str(idx)][i])
        dataset['Prophet_'+str(idx)][i] = dataset['Prophet_'+str(idx)][i-1]

2019-02-04 nan
2019-06-16 nan
2019-09-22 nan


C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [110]:
def normalize(target,y_max,y_min,y_mean):
    target_denorm = target.apply(lambda y: (y - y_mean)/(y_max - y_min))
    return  target_denorm

In [111]:
def de_normalize(test,y_max,y_min,y_mean):
    test_denorm = test.apply(lambda y: y * (y_max - y_min) + y_mean)
    return test_denorm

In [112]:
now_target = 'Price_'+str(idx)
y_max = np.max(dataset[now_target])
y_min = np.min(dataset[now_target])
y_mean = np.mean(dataset[now_target])

In [113]:
price_norm = normalize(dataset[now_target],y_max,y_min,y_mean)
LSTM1_norm = normalize(dataset['LSTM1_'+str(idx)],y_max,y_min,y_mean)
LSTM2_norm = normalize(dataset['LSTM2_'+str(idx)],y_max,y_min,y_mean)
LSTM2_fs_norm = normalize(dataset['LSTM2_fs_'+str(idx)],y_max,y_min,y_mean)
ARIMA_norm = normalize(dataset['ARIMA_'+str(idx)],y_max,y_min,y_mean)
Prophet_norm = normalize(dataset['Prophet_'+str(idx)],y_max,y_min,y_mean)

In [114]:
print('LSTM1: ',veg2[idx],mean_squared_error(price_norm, LSTM1_norm))
print('LSTM2: ',veg2[idx],mean_squared_error(price_norm, LSTM2_norm))
print('LSTM2_fs: ',veg2[idx],mean_squared_error(price_norm, LSTM2_fs_norm))
print('ARIMA: ',veg2[idx],mean_squared_error(price_norm, ARIMA_norm))
print('Prophet: ',veg2[idx],mean_squared_error(price_norm, Prophet_norm))


LSTM1:  木瓜-網室紅肉 0.0162953664467966
LSTM2:  木瓜-網室紅肉 0.04861633533796436
LSTM2_fs:  木瓜-網室紅肉 0.08644979756871189
ARIMA:  木瓜-網室紅肉 0.023787291933905517
Prophet:  木瓜-網室紅肉 0.1969652665364063


In [23]:
pd.concat([price_norm, LSTM1_norm, LSTM2_norm, LSTM2_fs_norm, ARIMA_norm, Prophet_norm], axis=1)

,Price_0,LSTM1_0,LSTM2_0,LSTM2_fs_0,ARIMA_0,Prophet_0
0,-0.039677,-0.035993,0.229757,-0.170205,-0.014926,0.354090
1,-0.044082,-0.030941,0.110268,-0.305913,-0.030933,0.366673
2,-0.000030,-0.022556,0.131489,-0.286490,-0.035659,0.372102
3,-0.017651,0.034608,0.295485,-0.069158,0.011595,0.355527
4,0.092481,0.045861,0.420451,0.311731,-0.007306,0.378064
...,...,...,...,...,...,...
293,0.330367,0.288500,0.450210,0.444230,0.389398,0.364730
294,0.295125,0.306346,0.509752,0.085951,0.365797,0.372186
295,0.259882,0.276976,0.450098,-0.002387,0.328032,0.358601
296,0.233451,0.259567,0.424470,0.048680,0.290263,0.383801


In [24]:
price_denorm = de_normalize(price_norm,y_max,y_min,y_mean)
LSTM1_denorm = de_normalize(LSTM1_norm,y_max,y_min,y_mean)
LSTM2_denorm = de_normalize(LSTM2_norm,y_max,y_min,y_mean)
LSTM2_fs_denorm = de_normalize(LSTM2_fs_norm,y_max,y_min,y_mean)
ARIMA_denorm = de_normalize(ARIMA_norm,y_max,y_min,y_mean)
Prophet_denorm = de_normalize(Prophet_norm,y_max,y_min,y_mean)

In [25]:
pd.concat([price_denorm, LSTM1_denorm, LSTM2_denorm, LSTM2_fs_denorm, ARIMA_denorm, Prophet_denorm], axis=1)

,Price_0,LSTM1_0,LSTM2_0,LSTM2_fs_0,ARIMA_0,Prophet_0
0,26.4,26.483630,32.516150,23.437027,26.961843,35.338524
1,26.3,26.598310,29.803760,20.356453,26.598490,35.624145
2,27.3,26.788643,30.285475,20.797358,26.491212,35.747380
3,26.9,28.086271,34.008186,25.730795,27.563889,35.371132
4,29.4,28.341724,36.844913,34.376972,27.134836,35.882735
...,...,...,...,...,...,...
293,34.8,33.849620,37.520428,37.384693,36.140011,35.580035
294,34.0,34.254734,38.872032,29.251760,35.604262,35.749286
295,33.2,33.588028,37.517890,27.246487,34.746996,35.440917
296,32.6,33.192840,36.936140,28.405700,33.889647,36.012947
